# 第3回 その1: 時系列データの可視化 (1)：移動平均と階差
時系列データの可視化手法である，移動平均と階差を実際にプログラムにより動作確認してみましょう。  
* 移動平均：時系列データの長期的な変動を可視化する。  
* 階差：時系列データの短期的な変動を可視化する。  

## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/03"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/03/
# 現在のフォルダの中身を表示
%ls

kishou.csv というデータが表示されていることを確認してください。

## ステップ1: 移動平均
気象データを読み込み，東京の気温データに対して移動平均を計算します。

まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`kishou.csv` を読み込み，東京の気温データを抽出してプロットします。  
この処理は第二回の 02_01_graph.ipynb で解説していますので，分からない方は復習してください。

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('kishou.csv', encoding='SHIFT-JIS')
# 表示
display(csv_data)

# 東京の気温を numpy用データ(ndarray型) として抽出する。
data_tokyo = csv_data.loc[:, '東京'].to_numpy()

# 抽出したデータをプロット
x = np.arange(np.size(data_tokyo))
plt.figure(figsize=(10,5))
plt.plot(x, data_tokyo, label='Original Data', color='b')
plt.xlabel('Day')
plt.ylabel('Temperature')
plt.legend()
plt.show()

さて，抽出したデータ `data_tokyo` に対して移動平均を求めます。  
ここでは各日に対して，前後15日のデータを用いて移動平均を求めています。  
(つまり K = 2*15 + 1 = 31項移動平均)  

In [ ]:
# データの要素数 (=366)
num_data = np.size(data_tokyo)
# 移動平均の計算結果の格納先を作成しておく(ここでは zeros 関数を使って， data_tokyo と同じサイズのゼロベクトルを作成)
moving_average = np.zeros(num_data)

# 前後いくつのサンプルを用いて平均値を求めるか
k = 15

for n in range(num_data):
  if n - k < 0:
    # 0日目より前のデータは平均計算に使用しない
    moving_average[n] = np.mean(data_tokyo[:n+k+1])
  elif n + k >= num_data:
    # データの最終日より後ろのデータは平均計算に使用しない
    moving_average[n] = np.mean(data_tokyo[n-k:])
  else:
    moving_average[n] = np.mean(data_tokyo[n-k : n+k+1])

計算した移動平均をプロットします。  
元のデータを青線で，移動平均を赤線で，重ねてプロットします。   
（グラフのラベル(凡例)の表示方法については第二回の02_01_graph.ipynb を参照）

In [ ]:
x = np.arange(np.size(data_tokyo))
plt.figure(figsize=(10,5))
plt.plot(x, data_tokyo, label='Original Data', color='b')
plt.plot(x, moving_average, label='Moving Average', color='r')
plt.xlabel('Day')
plt.ylabel('Temperature')
plt.legend()
plt.show()

さて，移動平均のパラメータである k を色々な値に変えて，結果がどうなるか確認してみましょう。  
パラメータを変えるたびに上記の移動平均の計算アルゴリズムを記述するのは面倒です。  
そこで，移動平均計算部分を関数として定義しておきましょう。  
関数の定義は  
```
def 関数名 (引数1, 引数2, ...):
  処理
  return 戻り値 (戻り値が無い場合のみ記述)
```
というように記述します。

In [ ]:
# 移動平均を計算する関数
def calc_moving_average(data, k):
  '''
     data: 移動平均を計算したいデータ
     k: 移動平均の計算に前後何サンプルの値を使用するか
  '''
  # データの要素数
  num_data = np.size(data)
  # 移動平均の計算結果の格納先を作成しておく(ここでは zeros 関数を使って， data_tokyo と同じサイズのゼロベクトルを作成)
  moving_average = np.zeros(num_data)
  for n in range(num_data):
    if n - k < 0:
      # 0日目より前のデータは平均計算に使用しない
      moving_average[n] = np.mean(data[:n+k+1])
    elif n + k >= num_data:
      # データの最終日より後ろのデータは平均計算に使用しない
      moving_average[n] = np.mean(data[n-k:])
    else:
      moving_average[n] = np.mean(data[n-k : n+k+1])
  
  return moving_average

（上のソースコードは，単に関数を定義しただけなので，実行しても何も出力されません。）

上で定義した関数 `calc_moving_average` を用いて，  
k を 1, 5, 15, 30 と変化させたときの移動平均をプロットしてみましょう。

In [ ]:
k_list = [1, 5, 15, 30]
color_list = ['r', 'g', 'y', 'm']

x = np.arange(np.size(data_tokyo))
plt.figure(figsize=(10,5))
# 元データのプロット
plt.plot(x, data_tokyo, label='Original Data', color='b')
# 4種類(k=1, 5, 15, 30)の移動平均を計算・プロット
for i in range(4):
  k = k_list[i]
  color = color_list[i]
  # 関数 calc_moving_average を使用して移動平均を計算
  moving_average = calc_moving_average(data_tokyo, k)
  # 移動平均のプロット
  plt.plot(x, moving_average, label='Moving Average: k='+str(k), color=color)
plt.xlabel('Day')
plt.ylabel('Temperature')
plt.legend()
plt.show()


kの値が大きくなるにつれて，計算された移動平均が滑らかになっているのが分かります。

## ステップ2: 階差
次に，階差を求めてみます。  
階差を求める関数を以下に定義します。

In [ ]:
# 階差を計算する関数
def calc_difference(data):
  '''
     data: 階差を計算したいデータ
  '''
  # データの要素数
  num_data = np.size(data)
  # 階差の計算結果の格納先を作成
  difference = np.zeros(num_data)
  for n in range(num_data):
    if n == 0:
      # n = 0 のときは階差=0とする。
      difference[n] = 0
    else:
      difference[n] = data[n] - data[n-1]
  
  return difference

定義した関数 `calc_difference` を用いて東京気温の階差を計算し，プロットしてみます。

In [ ]:
x = np.arange(np.size(data_tokyo))
difference = calc_difference(data_tokyo)
plt.figure(figsize=(10,5))
plt.plot(x, difference, label='Difference', color='b')
plt.xlabel('Day')
plt.ylabel('Temperature difference')
plt.legend()
plt.show()

東京と那覇の階差を比較してみましょう。

In [ ]:
x = np.arange(np.size(data_tokyo))
plt.figure(figsize=(10,5))

# 東京の階差を再度計算してプロット
difference_tokyo = calc_difference(data_tokyo)
plt.plot(x, difference_tokyo, label='Tokyo', color='b')

# 那覇のデータを抽出
data_naha = csv_data.loc[:, '那覇'].to_numpy()
# 那覇の階差を計算してプロット
difference_naha = calc_difference(data_naha)
plt.plot(x, difference_naha, label='Naha', color='r')

plt.xlabel('Day')
plt.ylabel('Temperature difference')
plt.legend()
plt.show()

上の結果から，那覇は東京と比べて気温の変化が小さいことが分かりました。